In [1]:
# Ensures that there is enough memory allocation for the model to load
import os
import torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

In [1]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
import json
import os 
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True) 
model.to("cuda:0")

/home/jovyan/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  8.03it/s]


LlavaNextForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPSdpaAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn

In [6]:
import re
# Path to the directory (library)
directory_path = "/home/jovyan/images"
# List all files in the directory
files = os.listdir(directory_path)

# Filter out directories and show only files
files = [file for file in files if os.path.isfile(os.path.join(directory_path, file))]

data = {}

# Print the list of files
for index, file in enumerate(files):
    # Optimize memory usage 
    torch.cuda.empty_cache()
    print(f"Processing image {file}")
    
    image_path = os.path.join(directory_path, file)
    image = Image.open(image_path)

    # Prompt for the Llava-hf
    prompt = f"""
    TASK : Object Character Recognition by Region
    Detect all text in the image.
    Do not include any header.
    Include only the text.
    """
    conversation = [
        {

          "role": "user",
          "content": [
              {"type": "text", "text": {prompt}},
              {"type": "image"},
            ],
        },
    ]
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

    inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda:0")

    # autoregressively complete prompt
    output = model.generate(**inputs, max_new_tokens=500)

    answer = processor.decode(output[0], skip_special_tokens=True)
 
    # Attains only the output and don't include the prompt 
    skip = len(prompt) - 6 
    answer = answer[skip:].split()
    
    data[file] = answer
    
    print(f"Successfully processed Image {file} , {index+1}/{len(files)} Images Processed")
     
with open('LlavaOCR.json' , 'w') as json_file:
        json.dump(data, json_file, indent=4)
   
print(f"All {len(files)} have been processed.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing image 588.JPG
Successfully processed Image 588.JPG , 1/265 Images Processed
Processing image 708.png


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Successfully processed Image 708.png , 2/265 Images Processed
Processing image 603.jpg
Successfully processed Image 603.jpg , 3/265 Images Processed
Processing image 546.jpg


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Successfully processed Image 552.jpg , 262/265 Images Processed
Processing image 658.jpg


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Successfully processed Image 658.jpg , 263/265 Images Processed
Processing image 719.png


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Successfully processed Image 719.png , 264/265 Images Processed
Processing image 579.jpg
Successfully processed Image 579.jpg , 265/265 Images Processed
All 265 have been processed.
